<a href="https://colab.research.google.com/github/Abhinaay/TextSummarization-and-Classification/blob/master/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementation of 2nd method for sentence to vector conversion.**

In [0]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=3423de46780e81affa839dc9a2448317806f0a30aa8dc3c28665cf8c7b6b87c7
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [0]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re
import bs4 as bs
import urllib.request

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
time: 1.87 s


In [0]:
scraped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Narendra_Modi')
article = scraped_data.read()

time: 212 ms


In [0]:
parsed_article = bs.BeautifulSoup(article,'lxml')
paragraphs = parsed_article.find_all('p')
article_text = ""
for p in paragraphs:
    article_text += p.text

time: 336 ms


In [0]:
# Sentence Tokenization
sentences = nltk.sent_tokenize(article_text)

time: 35.5 ms


In [0]:
# Download Glove Word Enbedding
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-01-10 10:53:14--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-01-10 10:53:14--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-01-10 10:53:15--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Preprocessing

In [0]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

time: 12.4 ms


In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

time: 61.4 ms


In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

time: 4.75 ms


In [0]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

time: 1.79 ms


In [0]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

time: 21.6 ms


In [0]:
# Vector Representation of Sentences
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

time: 219 ms


In [0]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

time: 24.7 ms


In [0]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

time: 3.04 ms


In [0]:
sim_mat.shape

(408, 408)

time: 2.68 ms


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

time: 1.34 ms


In [0]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [0]:
# Applying Text Rank Algorithm

time: 896 µs


In [0]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

time: 2.36 s


In [0]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

time: 2.29 ms


In [0]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

[50] Shortly before the war, Modi took part in a non-violent protest against the Indian government in New Delhi, for which he was arrested; this has been cited as a reason for Inamdar electing to mentor him.
[124] Modi framed the criticism of his government for human rights violations as an attack upon Gujarati pride, a strategy which led to the BJP winning two-thirds of the seats in the state assembly.
[88][89] The president of the state unit of the BJP expressed support for the bandh, despite such actions being illegal at the time.
[258][260][259]
The BJP election manifesto had also promised to deal with illegal immigration into India in the Northeast, as well as to be more firm in its handling of insurgent groups.
[26][71][72] However, he took a brief break from politics in 1992, instead establishing a school in Ahmedabad; friction with Shankersingh Vaghela, a BJP MP from Gujarat at the time, also played a part in this decision.
[131][132]
Questions about Modi's relationship with Mu